In [53]:
import json
import os

import geopandas
from dotenv import load_dotenv
from openai import OpenAI
from shapely.geometry import Polygon

In [59]:
# load land usage data
load_dotenv("../.env")

path = "../data/uso_suelos/"
filename = "05_region_valparaiso.shp"

# Load files
original_df = geopandas.read_file(
    os.path.join(path, filename)
)

# Get example polygon
df_demo = geopandas.read_file(
    "../data/demo_data/doc.kml"
)

In [60]:
# filter columns
selected_columns = ["USO_TIERRA", "USO", "NOM_REG", "NOM_COM", "geometry"]
df = original_df[selected_columns]

# parse data
df_kms = df.to_crs("EPSG:32633")
demo_geometry_kms = df_demo.to_crs("EPSG:32633")

In [62]:
# buscamos si el poligono demo se sobrepone en todos los poligonos restringidos
overlapping_geometries = df_kms[df_kms.geometry.overlaps(demo_geometry_kms.iloc[0].geometry)]
overlapping_geometries

,USO_TIERRA,USO,NOM_REG,NOM_COM,geometry
9378,"Ciudades, Pueblos, Zonas Industriales",Áreas Urbanas e Industriales,Región de Valparaíso,Casablanca,"POLYGON ((-7156556.944 -9400526.041, -7156539...."
9610,Terreno de Uso Agrícola,Terrenos Agrícolas,Región de Valparaíso,Casablanca,"POLYGON ((-7155146.725 -9399119.228, -7155144...."
9611,Terreno de Uso Agrícola,Terrenos Agrícolas,Región de Valparaíso,Casablanca,"POLYGON ((-7155463.727 -9400093.248, -7155472...."
9612,"Ciudades, Pueblos, Zonas Industriales",Áreas Urbanas e Industriales,Región de Valparaíso,Casablanca,"POLYGON ((-7156212.718 -9400702.359, -7156216...."
9631,Terreno de Uso Agrícola,Terrenos Agrícolas,Región de Valparaíso,Casablanca,"POLYGON ((-7156817.114 -9400229.853, -7156853...."
14753,Bosque Nativo Adulto Abierto,Bosques,Región de Valparaíso,Casablanca,"POLYGON ((-7157566.979 -9398655.896, -7157565...."
14754,Bosque Nativo Adulto Abierto,Bosques,Región de Valparaíso,Casablanca,"POLYGON ((-7157436.071 -9398342.399, -7157438...."
15402,Bosque Nativo Renoval Abierto,Bosques,Región de Valparaíso,Casablanca,"POLYGON ((-7154803.748 -9399627.106, -7154804...."
15551,Bosque Nativo Renoval Abierto,Bosques,Región de Valparaíso,Casablanca,"POLYGON ((-7157375.943 -9397436.767, -7157374...."
15580,Bosque Nativo Renoval Abierto,Bosques,Región de Valparaíso,Casablanca,"POLYGON ((-7155922.881 -9399123.606, -7155921...."


In [63]:
class LandUseAssistant:

    def __init__(self, dataframe: geopandas.GeoDataFrame, client: object):
        self.dataframe = dataframe
        self.client = client

    @staticmethod
    def system_prompt() -> str:
        system_prompt = """
        Eres un experto en uso de suelos y te han contratado para hacer la evaluación de impacto ambiental de un nuevo proyecto fotovoltaico en Chile.

        El usuario entregará la ubicación y los diferentes usos de suelo en un radio de 3 km.

        Tu trabajo es entregar una detallada evaluación de impacto ambiental del proyecto fotovoltaico en las áreas de diferentes usos de suelo cercanas.

        Para esto, primero debes entregar un resumen si encuentras algo crítico para rechazar el proyecto; si no, indica que todo está bien.

        En caso de encontrar algo crítico, debes entregar una evaluación detallada de los impactos ambientales y sociales del proyecto.

        El output será un JSON con dos campos:

        ```json
        {
            "resumen": "Resumen corto de la evaluación",
            "evaluacion": "Detalle de la evaluación"
        }
        ```
        
        Las categorías de uso de suelo que considerarás son las siguientes: áreas desprovistas de vegetación, praderas y matorrales, humedales, bosque.
        """
        return system_prompt

    @staticmethod
    def format_message(nearby_land_use: geopandas.GeoDataFrame) -> str:
        nearby_land_use_data = str(nearby_land_use.to_dict("records"))
        message = """
        Los tipos de uso de suelo dentro de un radio de 3 km de la ubicación del proyecto son:
        {nearby_land_use}
        """
        return message.replace("{nearby_land_use}", nearby_land_use_data)

    def evaluate_project(self, location: Polygon) -> dict:
        # Calculate the distance from the project location to the surrounding land use areas
        overlaps = df_kms[df_kms.geometry.overlaps(demo_geometry_kms.iloc[0].geometry)]

        print("Number of nearby land use areas: ", overlaps.shape[0])

        # Prepare the system prompt and user message for the model
        system_prompt = self.system_prompt()
        message = self.format_message(overlaps)

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": message}
        ]

        # Generate the response using the model
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            response_format={"type": "json_object"},
            temperature=0
        )

        return json.loads(response.choices[0].message.content)


In [64]:
park_assistant = LandUseAssistant(df_kms, OpenAI())

In [65]:
response = park_assistant.evaluate_project(demo_geometry_kms)

Number of nearby land use areas:  13


In [66]:
response

{'resumen': 'Se identificaron áreas de bosque nativo en las cercanías del proyecto fotovoltaico, lo cual podría generar impactos ambientales significativos.',
 'evaluacion': 'El proyecto fotovoltaico podría tener un impacto negativo en los bosques nativos cercanos, afectando la biodiversidad y el hábitat de especies locales. Se recomienda realizar un estudio detallado de los impactos ambientales y tomar medidas de mitigación, como la reforestación de áreas afectadas o la implementación de medidas para minimizar la fragmentación del hábitat.'}